Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.


In [5]:
import numpy as np
import pandas as pd
import math
import random
from openpyxl import Workbook

In [2]:
def Odleglosc(distance, trasa):
    suma = 0
    n = len(trasa)
    
    for i, current_city in enumerate(trasa):
        next_city = trasa[(i + 1) % n]  # Use modulo to handle the last element
        suma += distance[current_city][next_city]

    return suma

In [3]:
#swap
def Swap(trasa):
    trasa_temp = trasa.copy()
    #print(trasa)
    swap = np.random.choice(trasa_temp,2,replace=False)
    temp = trasa_temp[swap[0]]
    trasa_temp[swap[0]] = trasa_temp[swap[1]]
    trasa_temp[swap[1]] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

def Swap_tabu(trasa, miasto1, miasto2):
    trasa_temp = trasa.copy()
    #print(trasa)
    #swap = np.random.choice(trasa_temp,1)
    temp = trasa_temp[miasto1]
    trasa_temp[miasto1] = trasa_temp[miasto2]
    trasa_temp[miasto2] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

#insercja
def Insercja(trasa):
    trasa_temp = trasa.copy()
    insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[insercja[0]]
    trasa_temp = np.delete(trasa_temp,insercja[0])
    trasa_temp = np.insert(trasa_temp,insercja[1],temp)
    #print(trasa_temp)
    return trasa_temp

def Insercja_tabu(trasa, miasto1, miasto2):
    trasa_temp = trasa.copy()
    #insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[miasto1]
    trasa_temp = np.delete(trasa_temp,miasto1)
    trasa_temp = np.insert(trasa_temp,miasto2,temp)
    #print(trasa_temp)
    return trasa_temp    

def Zamiana(trasa):
    trasa_temp = trasa.copy()
    zamiana = np.random.choice(trasa_temp,2,replace=False)
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

def Zamiana_tabu(trasa, miasto1, miasto2):
    trasa_temp = trasa.copy()
    zamiana = [miasto1, miasto2]
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

In [4]:
distance = pd.read_excel("Dane_TSP_48.xlsx")
distance = distance.to_numpy()
distance = np.delete(distance,0,1)
distance

array([[   0, 1495,  381, ...,  488,  753, 1184],
       [1495,    0, 1135, ..., 1727, 1388,  661],
       [ 381, 1135,    0, ...,  640,  591,  810],
       ...,
       [ 488, 1727,  640, ...,    0,  520, 1242],
       [ 753, 1388,  591, ...,  520,    0,  801],
       [1184,  661,  810, ..., 1242,  801,    0]], dtype=int64)

In [44]:
colnames = ["metoda","liczba miast","trasa","odleglosc","ruch","liczba iteracji ruchu","liczba iteracji algorytmu"]
df_base = pd.read_excel("Solutions\\IHC.xlsx")
df_base.columns = colnames
metoda = "IHC"
rodzaj_dystansu = "{} miast".format(len(distance))

def IHC(distance, ruch, n_iter_ruch, n_iter):  # ruch = {Swap, Insercja, Zamiana}
    min_iter = []
    for i in range(n_iter):
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                if odl_new < odl:
                    odl = odl_new
                    trasa = trasa_new
        min_iter.append((metoda,rodzaj_dystansu,trasa,odl,ruch.__name__,n_iter,n_iter_ruch))
    return min_iter

wynik = IHC(distance,Zamiana,1000,100)
wynik.sort(key=lambda x: x[3])
wynik[0:10]

df = pd.DataFrame(wynik)
df.columns = colnames
df = pd.concat([df_base,df],ignore_index=True)
df.to_excel("Solutions\\IHC.xlsx",index=False)

In [41]:
def Geometric(T,alpha):
    return alpha*T

def Slow(T,alpha):
    return T/(1+alpha*T)

In [43]:
colnames = ["metoda","liczba miast","trasa","odleglosc","ruch","liczba iteracji ruchu","liczba iteracji algorytmu","temperatura startowa","rodzaj chlodzenia","parametr chlodzenia"]
df_base = pd.read_excel("Solutions\\SA.xlsx")
df_base.columns = colnames
metoda = "SA"
rodzaj_dystansu = "{} miast".format(len(distance))

def SA(distance, ruch, n_iter_ruch, n_iter, T_start, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((metoda,rodzaj_dystansu,trasa,odl,ruch.__name__,n_iter_ruch,n_iter,T_start,cooling.__name__,alpha))
    return min_iter

wynik = SA(distance,Swap,1000,100,100,Slow,0.1)
wynik.sort(key=lambda x: x[3])
wynik[0:10]

df = pd.DataFrame(wynik)
df.columns = colnames
df = pd.concat([df_base,df],ignore_index=True)
df.to_excel("Solutions\\SA.xlsx",index=False)

In [ ]:
# wersja SA gdzie warunkiem stopu jest temperatura a nie liczba iteracji 
# (chyba lepiej tak bo upewniamy sie ze temperatura jest niska a podajac liczbe iteracji mozemy skonczyc na wysokiej temperaturze)
# mozna tez dodac warunek stopu ze po np 100 iteracjach bez poprawy rozwiazania to koniec

def SA(distance, ruch, n_iter, T_start, T_stop, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        while(T > T_stop):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,100,100,0.01,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

In [ ]:
# funkcje pomocnicze do sprawdzenia ile iteracji odpowiada parametrowi T_stop
def T_stopToIter(T_start, T_stop, cooling, alpha):
    T = T_start
    i = 0
    while(T > T_stop):
        T = cooling(T,alpha)
        i += 1
    return i
def IterToT_stop(T_start, n_iter, cooling, alpha):
    T = T_start
    for i in range(n_iter):
        T = cooling(T,alpha)
    return T
print(T_stopToIter(100,0.01,Slow,0.1))
IterToT_stop(100,100,Slow,0.1)

algorytm genetyczny

In [85]:
def TournamentSelection(distance,population, elite_size=0):
    parents = []
    elite_count = int(len(population) * elite_size)
    elite = sorted(population, key=lambda x: Odleglosc(distance,x))[:elite_count]
    parents = elite.copy()  

    for _ in range(len(population)-elite_count):
        tournament = random.sample(population,5) # losuje 5 osobnikow z populacji
        tournament.sort(key=lambda x: Odleglosc(distance,x))
        parents.append(tournament[0])
    parents.sort(key=lambda x: Odleglosc(distance,x))
    return parents


In [50]:
def Ox(p1, p2):
    if not (isinstance(p1, list)): 
        p1 = p1.tolist()
    if not isinstance(p2, list): 
        p2 = p2.tolist()
    n = len(p1) 
    a = random.randint(0,n-1)
    b = random.randint(0,n-1)
    c1 = p1.copy()
    c2 = p2.copy()
    if a > b:                                                               # a musi byc mniejsze od b
        a,b = b,a
    section = np.append(np.arange(b,n),np.arange(a))                        # section = przedzial (b,n> + <0,a)  
    c1[0:a], c1[b:n] = ["x"]*(a-0),["x"]*(n-b)                                 # x dla: <0,a> U <b,n>,  <a,b> zostaje bez zmian
    c2[0:a], c2[b:n] = ["x"]*(a-0), ["x"]*(n-b)                                 # -//-
    iter_r = np.append(np.arange(b,n),np.arange(b))          # iter_r = <b,n> U <0,b> - tablica indeksow po których bedziemy...
    iter_p1_idx, iter_p2_idx = 0, 0 #iterowac szukając potencjalnych liczb w rodzicu które nie znajdują się jeszcze w potomku (czyli p2[iter_r[iter_p1_idx]])
    for i in section:                                                       # dla każdego indeksu w przedziale section (indeksow dla "x")
        while True:                                                         # wypelniamy kazde z "x" potomka1
            if p2[iter_r[iter_p1_idx]] not in c1:                           # jezeli liczba nie znajduje sie w potomku to ja wstawiamy
                c1[i] = p2[iter_r[iter_p1_idx]]
                iter_p1_idx+=1
                break
            else:                                                           # jeżeli liczba znajduje sie w potomku to iterujemy dalej po iter_r
                iter_p1_idx+=1
        while True:                                                         # wypelniamy kazde z "x" potomka2...
            if p1[iter_r[iter_p2_idx]] not in c2:
                c2[i] = p1[iter_r[iter_p2_idx]]
                iter_p2_idx+=1
                break
            else:
                iter_p2_idx+=1
    return c1, c2

In [90]:
colnames = ["metoda","liczba miast","trasa","odleglosc","mutacja(ruch)","liczba iteracji algorytmu","wielkość populacji","prawdopodobieństwo krzyżowania","prawdopodobieństwo krzyżowania","prawdopodobieństwo mutacji"]
df_base = pd.read_excel("Solutions\\GA.xlsx")
df_base.columns = colnames
metoda = "GA"
rodzaj_dystansu = "{} miast".format(len(distance))

def GA(distance, selection, mutation, n_iter, n_pop, p_cross, p_mut, elite_size=0, max_gen_no_improve=99999):
    best = 1e10
    gen_no_improve = 0
    population = []
    for _ in range(n_pop):
        trasa_rand = np.arange(len(distance))
        np.random.shuffle(trasa_rand)
        population.append(trasa_rand)
    for _ in range(n_iter):
        children = []
        parents = selection(distance,population,elite_size)
        for i in range(0, len(parents), 2): #krzyzowanie 
            parent1, parent2 = parents[i], parents[i+1]
            if random.random() < p_cross:
                child1, child2 = Ox(parent1, parent2)
                children.append(child1)
                children.append(child2)
            else:
                children.append(parent1)
                children.append(parent2)
        for i in range(len(children)): #mutacja
            if random.random() < p_mut:
                    children[i] = mutation(children[i])
        curr_best = Odleglosc(distance, min(children, key=lambda x: Odleglosc(distance,x))) #najlepszy osobnik z populacji
        print(curr_best)
        if curr_best >= best:
            gen_no_improve += 1
        else:
            gen_no_improve = 0
        best = curr_best
        population = children
        if gen_no_improve >= max_gen_no_improve:
            break
    for i in range(len(population)):
        population[i] = (metoda,rodzaj_dystansu,population[i],Odleglosc(distance,population[i]),selection.__name__,mutation.__name__,n_iter,n_pop,p_cross,p_mut)
    return (population)
for epoch in range(100):
    wynik = GA(distance,TournamentSelection, Swap, 100000, 1000, 0.9, 0.5, 0.1, 20)
    wynik
    df = pd.DataFrame(wynik)
    df.columns = colnames
    df = pd.concat([df_base,df],ignore_index=True)
    df.to_excel("Solutions\\GA.xlsx",index=False)

35939
35587
34438
33077
31656
31075
30896
29284
28347
27828
27669
27296
26555
25536
25427
24964
24785
24007
23656
23191
23191
23134
21796
21796
21721
20630
20630
20339
20339
19948
19948
19947
19725
19346
19346
19069
19346
18634
18634
18280
18280
17979
17979
17832
17754
17468
17349
17192
16956
16780
16772
16772
16661
16661
16661
16426
16381
16367
16314
16015
16015
15960
15602
15602
15602
15602
15469
15469
15435
15329
15329
15196
15100
14959
14920
14920
14890
14874
14874
14686
14686
14686
14686
14592
14578
14470
14470
14470
14470
14438
14417
14378
14378
14378
14371
14353
14261
14259
14015
14015
14015
14015
14015
14015
13820
13820
13820
13720
13720
13720
13720
13642
13642
13642
13642
13526
13526
13526
13526
13506
13506
13446
13446
13446
13446
13446
13446
13419
13415
13415
13415
13415
13388
13388
13388
13388
13388
13388
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
13379
36011
35874
33997
33427
33427
31808
31297
3123

PermissionError: [Errno 13] Permission denied: 'Solutions\\GA.xlsx'

In [54]:
#NN
colnames = ["metoda","liczba miast","trasa","odleglosc"]
df_base = pd.read_excel("Solutions\\NN.xlsx")
df_base.columns = colnames
metoda = "NN"
rodzaj_dystansu = "{} miast".format(len(distance))

min_odl = []
for n in range(len(distance)):
    potencjalne = np.arange(len(distance))
    trasa = np.zeros(len(distance),dtype=int)
    trasa[0] = potencjalne[n]
    potencjalne = np.delete(potencjalne,int(trasa[0]))

    for cur in range(1,len(trasa)):
        odleglosci = []
        for i in range(len(potencjalne)):
            trasa[cur] = potencjalne[i]
            odleglosci.append((potencjalne[i],Odleglosc(distance,trasa[0:cur+1])))
        odleglosci.sort(key=lambda x: x[1])
        trasa[cur] = odleglosci[0][0]
        potencjalne = np.delete(potencjalne,potencjalne==trasa[cur])
    min_odl.append((metoda,rodzaj_dystansu,trasa,Odleglosc(distance,trasa)))
    
min_odl.sort(key=lambda x:x[3])
wynik = min_odl
min_odl[:10]

df = pd.DataFrame(wynik)
df.columns = colnames
df = pd.concat([df_base,df],ignore_index=True)
df.columns = colnames
df.to_excel("Solutions\\NN.xlsx",index=False)

In [52]:
#lista tabu, tabu search
colnames = ["metoda","liczba miast","trasa","odleglosc","ruch","liczba iteracji ruchu","liczba iteracji algorytmu","dlugosc listy tabu",]
df_base = pd.read_excel("Solutions\\TS.xlsx")
df_base.columns = colnames
metoda = "TS"
rodzaj_dystansu = "{} miast".format(len(distance))

#dlugosc listy tabu jako parametr
def TS(distance, ruch, tabu_len, n_iter_ruch, n_iter):  # ruch = {Swap_tabu, Insercja_tabu, Zamiana_tabu}
    min_iter = []
    for i in range(n_iter):
        #losujemy trasę 
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        #inicjalizujemy listę tabu
        lista_tabu = []
        #rozpoczynamy pętlę która odpowiada za powtarzanie każdego ruchu
        for k in range(n_iter_ruch):
            #inicjalizujemy listę potencjalnych zamian
            potencjalne_zamiany = []
            #losujemy jedno stałe miasto w danym ruchu
            miasto = np.random.choice(trasa,1)
            #rozpoczynamy pętle która sprawdzi odległość dla ruchu pomiędzy wylosowanym stałym 
            #miastem a wszystkimi pozostałymi
            for j in range(len(distance)):
                trasa_temp = trasa.copy()
                #sprawdzamy czy kombinacja miast znajduje się na liście tabu
                if [miasto, j] not in lista_tabu and [j,miasto] not in lista_tabu:
                    #jeśli nie znajduje się na liście tabu to tworzymy nową trasę
                    trasa_temp = ruch(trasa_temp,miasto,j)
                    #do potencjalnych zamian dodajemy nową odległość oraz pomiędzy jakimi miastami wykonano ruch
                    potencjalne_zamiany.append([Odleglosc(distance,trasa_temp),miasto,j])
            #sortujemy wszystkie potencjalne ruchy według odległości
            potencjalne_zamiany.sort(key=lambda x:x[0])
            #wybieramy najlepszy możliwy ruch i wykonujemy go na pierwotnej trasie
            trasa = ruch(trasa,potencjalne_zamiany[0][1],potencjalne_zamiany[0][2])
            #dodajemy ruch do listy tabu
            lista_tabu.append([potencjalne_zamiany[0][1],potencjalne_zamiany[0][2]])
            #lista tabu zapełnia się z każdym ruchem, jeśli dojdzie do maksymalnej długości to usuwamy
            #najwcześniejszą wartość z listy
            if len(lista_tabu) == tabu_len:
                lista_tabu.remove(lista_tabu[0])
        #uzupełniamy listę potencjalnych tras
        min_iter.append([metoda,rodzaj_dystansu,trasa,Odleglosc(distance, trasa),ruch.__name__,n_iter_ruch,n_iter,tabu_len])
    return min_iter
wynik = TS(distance,Swap_tabu,5,100,100)
wynik.sort(key=lambda x: x[3])
wynik[0:10]

df = pd.DataFrame(wynik)
df.columns = colnames
df = pd.concat([df_base,df],ignore_index=True)
df.to_excel("Solutions\\TS.xlsx",index=False)
#zamiana chyba bez sensu

KeyboardInterrupt: 